- add tag to specify tissue and tool of models generated by isoquant and lyric from wing and leg of Lycaena helle
- combine models 
- quantify novelty
- run SQANTI
- include the resulting annotation to the overlapping analysis

### Add tag

In [5]:
models = ! ls ../data/longread_models/filtered/*corrected.gtf

In [30]:
import pyranges as pr
import pandas as pd

In [31]:
models

['../data/longread_models/filtered/Lh_leg_iqfree_sqanti_filter_corrected.gtf',
 '../data/longread_models/filtered/Lh_leg_lyric_sqanti_filter_corrected.gtf',
 '../data/longread_models/filtered/Lh_wing_iqfree_sqanti_filter_corrected.gtf',
 '../data/longread_models/filtered/Lh_wing_lyric_sqanti_filter_corrected.gtf']

In [34]:
for m in models:
    
    tool = m.split('/')[-1].split('_')[2]
    tissue = m.split('/')[-1].split('_')[1]
    out_file = m.replace('.gtf', '_plusTAG.gtf')
    
    pr_obj = pr.read_gtf(m)
    df_obj = pr_obj.df
    df_obj['tool'] = tool
    df_obj['tissue'] = tissue
    
    modified_pr_obj = pr.PyRanges(df_obj)
    modified_pr_obj.to_gtf(out_file)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to

### Merge models
This is will be done inside the agat container

```
docker run --platform linux/amd64 -v $PWD:$PWD -w $PWD -ti quay.io/biocontainers/agat:1.4.0--pl5321hdfd78af_0

agat_sp_merge_annotations.pl \
        --gff filtered/Lh_leg_iqfree_sqanti_filter_corrected_plusTAG.gtf \
        --gff filtered/Lh_leg_lyric_sqanti_filter_corrected_plusTAG.gtf \
        --gff filtered/Lh_wing_iqfree_sqanti_filter_corrected_plusTAG.gtf \
        --gff filtered/Lh_wing_lyric_sqanti_filter_corrected_plusTAG.gtf \
        --out combined_models/Lh_leg_wing_iqfree_lyric_merged_plusTAG.gff

```

### Refine annotation

This consist of rename genes and child feature so they follow a coherent pattern instead of mix of lyric and isoquant names. 

So far the most promising utility to do that is `agat_sq_manage_ID.pl`.  
Still the gene_ID and all the other feature remain untached and I'd like to fix them as well.

`agat_sp_manage_ID.pl` is not an automatic good option as it gives sequntial names to all feature so that gene1,trans2,exon3, gene4 etc. 